In [ ]:
# =============================================================================
# EXPERIMENT: [DESCRIBE YOUR EXPERIMENT HERE]
# Date: 21-11-2025 avond 22:23 started
# Description: Logistic Regression 
# =============================================================================

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import warnings
from datetime import datetime
from pathlib import Path

In [9]:
# =============================================================================
# 1. LOAD PREPROCESSED DATA (no need to preprocess again!)
# =============================================================================
print("Loading preprocessed data...")
X = pd.read_pickle('../../data/processed/X_train_processed.pkl')
y = pd.read_pickle('../../data/processed/y_train.pkl')
X_test = pd.read_pickle('../../data/processed/X_test_processed.pkl')
test_ids = pd.read_pickle('../../data/processed/test_ids.pkl')

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

Loading preprocessed data...
X shape: (20885, 55)
y shape: (20885,)


In [10]:
# =============================================================================
# 2. DEFINE EXPERIMENT PARAMETERS
# =============================================================================
EXPERIMENT_NAME = "logreg_C10_l2"
MODEL_DESCRIPTION = "Logistic Regression with C=10, L2 penalty, balanced class weights"

In [11]:
# =============================================================================
# 3. TRAIN MODEL
# =============================================================================
from sklearn.model_selection import StratifiedKFold, cross_val_score


print(f"\n{'='*70}")
print(f"EXPERIMENT: {EXPERIMENT_NAME}")
print(f"{'='*70}")

model = LogisticRegression(
    C=10,                      # ← MODIFY THIS
    penalty='l2',              # ← MODIFY THIS
    solver='lbfgs',
    max_iter=1000,
    random_state=42,
    class_weight='balanced'
)

# Cross-validation for reliable performance estimate
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=cv, scoring='roc_auc', n_jobs=-1)

print(f"\nCross-Validation ROC-AUC: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
print(f"Fold scores: {[f'{s:.4f}' for s in cv_scores]}")

# Train on full training set
model.fit(X, y)

# Training performance (sanity check)
y_pred_train = model.predict_proba(X)[:, 1]
train_auc = roc_auc_score(y, y_pred_train)
print(f"Training ROC-AUC: {train_auc:.4f}")


EXPERIMENT: logreg_C10_l2

Cross-Validation ROC-AUC: 0.8396 ± 0.0077
Fold scores: ['0.8436', '0.8308', '0.8522', '0.8335', '0.8378']
Training ROC-AUC: 0.8455


In [13]:
# =============================================================================
# 4. GENERATE PREDICTIONS
# =============================================================================
test_proba = model.predict_proba(X_test)[:, 1]

In [20]:
# =============================================================================
# 5. CREATE SUBMISSION FILE
# =============================================================================
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
submission_filename = f"../../outputs/predictions/logistic_regression/{EXPERIMENT_NAME}_{timestamp}.csv"

submission = pd.DataFrame({
    'icustay_id': test_ids,
    'prediction': test_proba
})
submission.to_csv(submission_filename, index=False)
print(f"\n✓ Submission saved: {submission_filename}")


✓ Submission saved: ../../outputs/predictions/logistic_regression/logreg_C10_l2_20251121_2351.csv


In [16]:
# =============================================================================
# 6. LOG EXPERIMENT (CRITICAL FOR TRACKING!)
# =============================================================================
experiment_log = {
    'timestamp': timestamp,
    'experiment_name': EXPERIMENT_NAME,
    'model_type': 'LogisticRegression',
    'cv_roc_auc_mean': cv_scores.mean(),
    'cv_roc_auc_std': cv_scores.std(),
    'train_roc_auc': train_auc,
    'kaggle_public_score': None,  # Fill in manually after submission
    'kaggle_private_score': None,
    'parameters': str(model.get_params()),
    'description': MODEL_DESCRIPTION,
    'submission_file': submission_filename
}

# Append to experiment log
log_file = '../experiment_log.csv'
log_df = pd.DataFrame([experiment_log])

import os
if os.path.exists(log_file):
    log_df.to_csv(log_file, mode='a', header=False, index=False)
else:
    log_df.to_csv(log_file, index=False)

print(f"✓ Experiment logged to {log_file}")

✓ Experiment logged to ../experiment_log.csv


In [19]:
# =============================================================================
# 7. SAVE MODEL (OPTIONAL)
# =============================================================================
import pickle


model_filename = f"../../outputs/models/{EXPERIMENT_NAME}_{timestamp}.pkl"
with open(model_filename, 'wb') as f:
    pickle.dump(model, f)
print(f"✓ Model saved: {model_filename}")

print(f"\n{'='*70}")
print("EXPERIMENT COMPLETE!")
print(f"{'='*70}")
print(f"\nNext steps:")
print(f"1. Upload {submission_filename} to Kaggle")
print(f"2. Record Kaggle score in {log_file}")
print(f"3. Copy this notebook and modify for next experiment")

✓ Model saved: ../../outputs/models/logreg_C10_l2_20251121_2346.pkl

EXPERIMENT COMPLETE!

Next steps:
1. Upload ../../outputs/predictions/logreg_C10_l2_20251121_2346.csv to Kaggle
2. Record Kaggle score in ../experiment_log.csv
3. Copy this notebook and modify for next experiment
